In [853]:
'''The Jupyter notebook is a web-based notebook environment for interactive computing. 
The Jupyter Notebook is an open source web application that you can use to create and share documents that 
contain live code, equations, visualizations, and text.
'''
print(f"** es-search-guard")
'''  Search Guard Is An Open Source Security Plugin For Elasticsearch  '''

** es-search-guard


'  Search Guard Is An Open Source Security Plugin For Elasticsearch  '

In [854]:
#pip install elasticsearch==7.13.0

In [855]:
from elasticsearch import Elasticsearch
import os
import json
import sys
import pandas as pd
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

In [856]:
sys.path.append(r"C://Users/euiyoung.hwang/Git_Workspace/ELK-Stack-Upgrade/jupyter-workflow/")

In [857]:
# import pytest
import ipytest

In [858]:
''' To begin, you need to install pytest and ipytest, a tool designed to run pytest tests directly in Jupyter. Execute the following in a Jupyter cell '''
ipytest.autoconfig()

In [859]:
''' pip install python-dotenv'''
load_dotenv() # will search for .env file in local folder and load variables 

True

In [860]:
''' The "Basic" HTTP authentication scheme is defined in RFC 7617, which transmits credentials as user ID/password pairs, encoded using base64 '''
''' Basic Authentication is a method for an HTTP user agent (e.g., a web browser) to provide a username and password when making a request. '''
''' You can send the authorization header when making requests and accessing to ES Cluster based on Search-Guard'''
''' Basic authentication is a very simple authentication scheme that is built into the HTTP protocol. The client sends HTTP requests with the Authorization header that contains the Basic word '''
def get_headers():
    ''' Elasticsearch Header '''
    return {'Content-type': 'application/json', 'Authorization' : '{}'.format(os.getenv('BASIC_AUTH')), 'Connection': 'close'}

### User Authorzation Check via ES cluster with no X-pack
* ES Version : ES v.5.6.4
* X-pack : Not Installed

In [861]:
''' Create ES Instance once user credential was added into headers. If it is http-based, even if the header value is sent, it is ignored. '''
''' https://www.debugbear.com/basic-auth-header-generator '''
''' Call API to DEV ES Cluster '''
es = Elasticsearch(hosts=os.getenv('ES_DEV_HOST'), headers=get_headers(), timeout=5,  verify_certs=False)

In [862]:
print(es.cat.health())

1736217563 21:39:23 supplychain-logging-dev green 4 4 1288 643 0 0 0 0 - 100.0%



In [863]:
print(json.dumps(es.info(), indent=2))

{
  "name": "supplychain-logging-dev-node-1",
  "cluster_name": "supplychain-logging-dev",
  "cluster_uuid": "uPGEwn_cTKOdfg5x32jHfQ",
  "version": {
    "number": "5.6.4",
    "build_hash": "8bbedf5",
    "build_date": "2017-10-31T18:55:38.105Z",
    "build_snapshot": false,
    "lucene_version": "6.6.1"
  },
  "tagline": "You Know, for Search"
}


In [864]:
print(f"es version : {es.info()['version']['number']}")

es version : 5.6.4


In [865]:
df = pd.DataFrame.from_dict([es.cluster.health()])
print(json.dumps(es.cluster.health(), indent=2))
display(df)

{
  "cluster_name": "supplychain-logging-dev",
  "status": "green",
  "timed_out": false,
  "number_of_nodes": 4,
  "number_of_data_nodes": 4,
  "active_primary_shards": 643,
  "active_shards": 1288,
  "relocating_shards": 0,
  "initializing_shards": 0,
  "unassigned_shards": 0,
  "delayed_unassigned_shards": 0,
  "number_of_pending_tasks": 0,
  "number_of_in_flight_fetch": 0,
  "task_max_waiting_in_queue_millis": 0,
  "active_shards_percent_as_number": 100.0
}


,cluster_name,status,timed_out,number_of_nodes,number_of_data_nodes,active_primary_shards,active_shards,relocating_shards,initializing_shards,unassigned_shards,delayed_unassigned_shards,number_of_pending_tasks,number_of_in_flight_fetch,task_max_waiting_in_queue_millis,active_shards_percent_as_number
0,supplychain-logging-dev,green,False,4,4,643,1288,0,0,0,0,0,0,0,100.0


### User Authentication Check via ES cluster with Search Guard
* ES Version : ES v.8.12
* X-pack : Search Guard for ES v.8.12

In [866]:
import base64

In [867]:
''' https://www.debugbear.com/basic-auth-header-generator '''
''' CASE#1 - Check when headers don't have basic auth info '''
es_search_guard = Elasticsearch(hosts=os.getenv('ES_DEV_V8_HOST'), headers=None, timeout=5,  verify_certs=False)

In [868]:
''' Getting error 'AuthenticationException(401, 'Unauthorized') '''
try:
    print(es_search_guard.cat.health())
except Exception as e:
    print(e)

AuthenticationException(401, 'Unauthorized')


In [869]:
''' test base64 encode '''
def base64_encode_for_search_guard(id_pass):
    ''' format -> <id>:<password> '''
    encoded = '{}'.format(base64.b64encode(id_pass.encode('utf-8')).decode())
    # print(encoded)
    return encoded

In [870]:
''' test base64 decode '''
def base64_decode_for_search_guard(id_pass):
    ''' format -> <id>:<password> '''
    return base64.b64decode(id_pass).decode('utf-8')

In [871]:
''' Generate base64 encode for id:pasword when making request to ES cluster with X-pack '''
encoded = base64_encode_for_search_guard('test:1')
print(f"encoded test : {encoded}")
decoded = base64_decode_for_search_guard(encoded)
print(f"decoded test : {decoded}")

encoded test : dGVzdDox
decoded test : test:1


In [872]:
''' https://www.debugbear.com/basic-auth-header-generator '''
''' CASE#2 - Check when headers have wrong basic auth info '''
es_search_guard = Elasticsearch(hosts=os.getenv('ES_DEV_V8_HOST'), 
                                headers={
                                    'Content-type': 'application/json', 
                                    'Authorization' : 'Basic {}'.format(base64_encode_for_search_guard('test:1')), 
                                    'Connection': 'close'
                                }, 
                                timeout=5,  
                                verify_certs=False)

In [873]:
''' Getting error 'AuthenticationException(401, 'Unauthorized') '''
try:
    print(es_search_guard.cat.health())
except Exception as e:
    print(e)

AuthenticationException(401, 'Unauthorized')


In [874]:
# print(get_headers())

In [875]:
''' https://www.debugbear.com/basic-auth-header-generator '''
''' CASE#3 - Check when headers have correct basic auth info. Before testing. we need to add a new user with right permission in ES v.8 Cluster '''
''' Call to Dev ES Cluster based on ES v.8.12 and Search-Guard as X-pack '''
es_search_guard = Elasticsearch(hosts=os.getenv('ES_V8_HOST'), headers=get_headers(), timeout=5,  verify_certs=False)

In [876]:
''' Getting the response correctly '''
try:
    print(es_search_guard.cat.health())
    df = pd.DataFrame.from_dict([es_search_guard.cluster.health()])
    print(json.dumps(es_search_guard.cluster.health(), indent=2))
    display(df)
except Exception as e:
    print(e)

1736217565 02:39:25 supplychain-logging-es8-dev green 3 3 590 294 0 0 0 0 - 100.0%

{
  "cluster_name": "supplychain-logging-es8-dev",
  "status": "green",
  "timed_out": false,
  "number_of_nodes": 3,
  "number_of_data_nodes": 3,
  "active_primary_shards": 294,
  "active_shards": 590,
  "relocating_shards": 0,
  "initializing_shards": 0,
  "unassigned_shards": 0,
  "delayed_unassigned_shards": 0,
  "number_of_pending_tasks": 0,
  "number_of_in_flight_fetch": 0,
  "task_max_waiting_in_queue_millis": 0,
  "active_shards_percent_as_number": 100.0
}


,cluster_name,status,timed_out,number_of_nodes,number_of_data_nodes,active_primary_shards,active_shards,relocating_shards,initializing_shards,unassigned_shards,delayed_unassigned_shards,number_of_pending_tasks,number_of_in_flight_fetch,task_max_waiting_in_queue_millis,active_shards_percent_as_number
0,supplychain-logging-es8-dev,green,False,3,3,294,590,0,0,0,0,0,0,0,100.0


In [877]:
print(f"es version : {es_search_guard.info()['version']['number']}")

es version : 8.12.2


In [878]:
''' Search Guard retrieves the user’s credentials from the request. '''
''' Search Guard authorizes the user by retrieving a list of the user’s roles from the configured authorization backend '''
print(f"** Role based User Permission check")

** Role based User Permission check
